# Создание модели, предлагающей оптимальный тарифный план
В данном проекте мы рассмотрим несколько моделей, способных спрогнозировать выбор того или иного тарифного плана на основе пользовательских предпочтений. Будет рассмотрена задача классификации в контексте двух тарифов SMART и ULTRA.

# Содержание

## [Загрузка данных](#step_1)
## [Делим выборки](#step_2)
## [Дерево решений](#step_3)
### [Промежуточный результат](#step_4)
## [Случайный лес](#step_5)
### [Новое предположение](#step_6)
### [Промежуточный результат](#step_7)
## [Логистическая регрессия](#step_8)
### [Промежуточный результат](#step_9)
## [Проверка моделей на тестовой выборке](#step_10)
### [Результаты](#step_11)
## [Проверка модели на адекватность](#step_12)
### [Результат](#step_13)
## [Градиентный бустинг над решающими деревьями](#step_14)
### [Новый претендент на лидерство](#step_15)
## [Итог](#step_16)

## Загрузим данные и посмотрим на них
<a id='step_1'></a>

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
df = pd.read_csv(r'C:\Users\Семен\Downloads\users_behavior.csv')
display(df.head(20))
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Итак, с данными все в порядке. В качестве `features` будем использовать `calls`, `minutes`, `messages`, `mb_used`. Наш `target` - это `is_ultra`.

## Получение обучающей, тестовой и валидационной выборок, отделение features от target
<a id='step_2'></a>

In [2]:
from sklearn.model_selection import train_test_split
df_train, df_test_valid = train_test_split(df, test_size = 0.4, random_state = 12345)
df_test, df_valid = train_test_split(df_test_valid, test_size = 0.5, random_state = 12345)

features_train = df_train.drop('is_ultra', axis = 1).copy()
target_train = df_train['is_ultra'].copy()

features_test = df_test.drop('is_ultra', axis = 1).copy()
target_test = df_test['is_ultra'].copy()

features_valid = df_valid.drop('is_ultra', axis = 1).copy()
target_valid = df_valid['is_ultra'].copy()

Проверим размер получившихся выборок

In [3]:
print('Размер обучающей выборки:', df_train.size)
print('Размер валидационной выборки:', df_valid.size)
print('Размер тестовой выборки:', df_test.size)
train_share = round((df_train.size / df.size), 2)
valid_share = round(df_valid.size / df.size, 2)
test_share = round(df_test.size / df.size, 2)
print('Соотношение', train_share, ':', valid_share, ':', test_share)

Размер обучающей выборки: 9640
Размер валидационной выборки: 3215
Размер тестовой выборки: 3215
Соотношение 0.6 : 0.2 : 0.2


Все верно, разделили правильно, дважды использовав `train_test_split`. Перейдем непосредственно к созданию моделей и их сравнению.
## Дерево решений
<a id='step_3
       '></a>

In [4]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn import tree
best_tree_valid = None
best_accuracy_valid = 0
best_tree_train = None
best_accuracy_train = 0
best_depth_valid = 0
best_depth_train = 0
for depth in range(1, 15):
    model = DecisionTreeClassifier(max_depth = depth, random_state = 12345)
    model.fit(features_train, target_train)
    predictions_train = model.predict(features_train)
    predictions_valid = model.predict(features_valid)
    accuracy_train = accuracy_score(predictions_train, target_train)
    accuracy_valid = accuracy_score(predictions_valid, target_valid)
    print('Для max_depth =', depth, ' accuracy по обучающей выборке равно', round(accuracy_train, 3), ' accuracy по валидационнной выборке равно', round(accuracy_valid, 3))
    if accuracy_train > best_accuracy_train:
        best_accuracy_train = accuracy_train
        best_tree_train = model
        best_depth_train = depth
    if accuracy_valid > best_accuracy_valid:
        best_accuracy_valid = accuracy_valid
        best_tree_valid = model
        best_depth_valid = depth
print(' ')
print('Лучшее дерево по валидационной выборке выборке имеет параметр max_depth =', best_depth_valid, ' и accuracy по валидационной выборке, равное', round(best_accuracy_valid, 4))
print('Лучшее дерево по обучающей выборке выборке имеет параметр max_depth =', best_depth_train, ' и accuracy по обучающей выборке, равное', round(best_accuracy_train, 4))
best_tree = best_tree_valid

Для max_depth = 1  accuracy по обучающей выборке равно 0.758  accuracy по валидационнной выборке равно 0.736
Для max_depth = 2  accuracy по обучающей выборке равно 0.788  accuracy по валидационнной выборке равно 0.774
Для max_depth = 3  accuracy по обучающей выборке равно 0.808  accuracy по валидационнной выборке равно 0.779
Для max_depth = 4  accuracy по обучающей выборке равно 0.811  accuracy по валидационнной выборке равно 0.774
Для max_depth = 5  accuracy по обучающей выборке равно 0.82  accuracy по валидационнной выборке равно 0.784
Для max_depth = 6  accuracy по обучающей выборке равно 0.838  accuracy по валидационнной выборке равно 0.776
Для max_depth = 7  accuracy по обучающей выборке равно 0.856  accuracy по валидационнной выборке равно 0.799
Для max_depth = 8  accuracy по обучающей выборке равно 0.863  accuracy по валидационнной выборке равно 0.793
Для max_depth = 9  accuracy по обучающей выборке равно 0.881  accuracy по валидационнной выборке равно 0.781
Для max_depth = 10  

### Промежуточный результат
Как видим, `accuracy` на лучшей модели решающего дерева достигается при максимальной глубине равной 7. Далее показатель по обучающей выборке возрастает, но по валидационной убывает - происхдит переобучение. Само дерево распечатывать не будем, поскольку оно попросту нечитаемо.

## Случайный лес
<a id='step_4'></a>

In [5]:
from sklearn.ensemble import RandomForestClassifier
best_forest_valid = None
best_accuracy_valid_forest = 0
best_forest_train = None
best_accuracy_train_forest = 0
for est in range(10, 51, 5):
    for depth_forest in range(1, 15):
        forest = RandomForestClassifier(random_state = 12345, max_depth = depth_forest, n_estimators = est)
        forest.fit(features_train, target_train)
        predictions_valid = forest.predict(features_valid)
        accuracy_valid_forest = accuracy_score(predictions_valid, target_valid)
        predictions_train = forest.predict(features_train)
        accuracy_train_forest = accuracy_score(predictions_train, target_train)
        if accuracy_train_forest > best_accuracy_train_forest:
            best_accuracy_train_forest = accuracy_train_forest
            best_est_train = est
            best_depth_forest_train = depth_forest
            best_forest_train = forest
        if accuracy_valid_forest > best_accuracy_valid_forest:
            best_accuracy_valid_forest = accuracy_valid_forest
            best_est_valid = est
            best_depth_forest_valid = depth_forest
            best_forest_valid = forest
print('Лучший случайный лес по валидационной выборке имеет параметр n_estimators = ', best_est_valid, ' max_depth =', best_depth_forest_valid, ' и значение accuracy =', round(best_accuracy_valid_forest, 3))   
print('Лучший случайный лес по обучающей выборке имеет параметр n_estimators = ', best_est_train, ' max_depth =', best_depth_forest_train, ' и значение accuracy =', round(best_accuracy_train_forest, 3))  

Лучший случайный лес по валидационной выборке имеет параметр n_estimators =  10  max_depth = 9  и значение accuracy = 0.813
Лучший случайный лес по обучающей выборке имеет параметр n_estimators =  50  max_depth = 14  и значение accuracy = 0.943


### Промежуточный результат и дополнительная проверка
<a id='step_5'></a>
Как видим, accuracy лучшего леса по валидационной выборке больше, чем то, что нам дало решающее дерево. А можно ли улучшить результат? Попробуем это сделать, рассмотрев деревья с `n_estimators` от 6 до 12 и max_depth от 6 до 13.

In [6]:
best_accuracy_train_forest_n = 0
best_accuracy_valid_forest_n = 0
for est in range(6, 13):
    for depth_forest in range(6, 14):
        forest = RandomForestClassifier(random_state = 12345, max_depth = depth_forest, n_estimators = est)
        forest.fit(features_train, target_train)
        predictions_valid = forest.predict(features_valid)
        accuracy_valid_forest = accuracy_score(predictions_valid, target_valid)
        predictions_train = forest.predict(features_train)
        accuracy_train_forest = accuracy_score(predictions_train, target_train)
        if accuracy_train_forest > best_accuracy_train_forest_n:
            best_accuracy_train_forest_n = accuracy_train_forest
            best_est_train_n = est
            best_depth_forest_train_n = depth_forest
        if accuracy_valid_forest > best_accuracy_valid_forest_n:
            best_accuracy_valid_forest_n = accuracy_valid_forest
            best_est_valid_n = est
            best_depth_forest_valid_n = depth_forest
print('Лучший случайный лес по валидационной выборке имеет параметр n_estimators = ', best_est_valid_n, ' max_depth =', best_depth_forest_valid_n, ' и значение accuracy =', round(best_accuracy_valid_forest_n, 3))   
print('Лучший случайный лес по обучающей выборке имеет параметр n_estimators = ', best_est_train_n, ' max_depth =', best_depth_forest_train_n, ' и значение accuracy =', round(best_accuracy_train_forest_n, 3))  

Лучший случайный лес по валидационной выборке имеет параметр n_estimators =  8  max_depth = 9  и значение accuracy = 0.813
Лучший случайный лес по обучающей выборке имеет параметр n_estimators =  8  max_depth = 13  и значение accuracy = 0.925


### Результаты дополнительной проверки
<a id='step_6'></a>
Проверка выдала довольно странный результат: то же самое значение accuracy, но другое значение `n_estimators`. Может дело в округлении? Посмотрим. 

In [7]:
forest_1 = RandomForestClassifier(n_estimators = 8, max_depth = 9, random_state = 12345)
forest_2 = RandomForestClassifier(n_estimators = 10, max_depth = 9, random_state = 12345)

forest_1.fit(features_train, target_train)
forest_2.fit(features_train, target_train)

predictions_1 = forest_1.predict(features_valid)
predictions_2 = forest_2.predict(features_valid)

accuracy_1 = accuracy_score(predictions_1, target_valid)
accuracy_2 = accuracy_score(predictions_2, target_valid)
if accuracy_1 > accuracy_2:
    print('Лучшая модель с n_estimators = 8 и max_depth = 9. Не зря проверяли гипотезу!')
    best_forest = forest_1
elif accuracy_1 == accuracy_2:
    print('Модели одинаково хороши! Интересно...')
    best_forest = forest_1
else:
    print('Лучшая модель с n_estimators = 10 и max_depth = 9. Но хорошо, что проаерили гипотезу!')
    best_forest = forest_2

Модели одинаково хороши! Интересно...


## Логистическая регрессия
<a id='step_7'></a>

In [8]:
from sklearn.linear_model import LogisticRegression
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
best_accuracy_lreg = 0
for solver in solvers:
    l_reg_model = LogisticRegression(random_state = 12345, solver = solver)
    l_reg_model.fit(features_train, target_train)
    lreg_predictions = l_reg_model.predict(features_train)
    accuracy_lreg = accuracy_score(lreg_predictions, target_train)
    print('При типе solver', solver, ' модель логистической регрессии дает accuracy =', round(accuracy_lreg, 3))
    if accuracy_lreg > best_accuracy_lreg:
        best_accuracy_lreg = accuracy_lreg
        best_solver = solver
        best_lreg = l_reg_model
print(' ')
print('Лучшая модель логистической регресии наблюдается при типе solver', best_solver, ' и дает дает accuracy =', round(best_accuracy_lreg, 3))

C:\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


При типе solver newton-cg  модель логистической регрессии дает accuracy = 0.753
При типе solver lbfgs  модель логистической регрессии дает accuracy = 0.713
При типе solver liblinear  модель логистической регрессии дает accuracy = 0.751
При типе solver sag  модель логистической регрессии дает accuracy = 0.692
При типе solver saga  модель логистической регрессии дает accuracy = 0.692
 
Лучшая модель логистической регресии наблюдается при типе solver newton-cg  и дает дает accuracy = 0.753


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


### Промежуточные результаты
<a id='step_8'></a>
Так, мы получили лучшее значение `accuracy`, которое нам может дать логистическая регресиия, и оно равно 0,753. Не густо!

## Проверка моделей на тестовой выборке
<a id='step_9'></a>

In [9]:
tree_predictions = best_tree.predict(features_test)
accuracy_tree = accuracy_score(tree_predictions, target_test)

forest_predictions = best_forest.predict(features_test)
accuracy_forest = accuracy_score(forest_predictions, target_test)

lreg_predictions = best_lreg.predict(features_test)
accuracy_lreg = accuracy_score(lreg_predictions, target_test)

print('Accuracy лучшего решающего дерева по тестовой выборке равно', accuracy_tree)
print('Accuracy лучшего случайного леса по тестовой выборке равно', accuracy_forest)
print('Accuracy лучшей логистической регресии по тестовой выборке равно', accuracy_lreg)

Accuracy лучшего решающего дерева по тестовой выборке равно 0.7822706065318819
Accuracy лучшего случайного леса по тестовой выборке равно 0.7791601866251944
Accuracy лучшей логистической регресии по тестовой выборке равно 0.7558320373250389


### Результаты
<a id='step_10'></a>
Интересно, что по валидационной выборке выигрывал случайный лес, а по тестовой с небольшим отрывам в лидеры выбился случайный лес. В спорной ситуации вспомним, что важна еще и скорость работы, а у случайный лес такой медленный. Лучшей моделью признаем решающее дерево.

## Проверка модели на адекватность 
<a id='step_12'></a>

In [10]:
df_groups = df_train.groupby('is_ultra').count()
df_groups
length = len(df_train)
max_class = df_groups.max().max()
print('Accuracy константного предсказания равно', max_class / length)

Accuracy константного предсказания равно 0.6924273858921162


### Результаты проверки
<a id='step_13'></a>
Видим, что выбранная лучшей модель предсказывает лучше лучшего константного предсказания. Отлично, модель прошла проверку на адекватность.

## Градиентный бустинг над решающими деревьями
<a id='step_14'></a>
Я попытаюсь, но ничего не гарантирую :)

In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
df_train['predictions'] = df_train['is_ultra'].mean()
df_valid['predictions'] = df_valid['is_ultra'].mean()
n = 170
nu = 0.1
best_acc = 0
trees = []
for i in range(n):
    df_train['residuals'] = df_train['is_ultra'] - df_train['predictions']
    tree = DecisionTreeRegressor(max_depth = 1)
    tree.fit(features_train, df_train['residuals'])
    df_train['predictions'] += nu * tree.predict(features_train)
    real_preds_train = (df_train['predictions']+0.5).astype('int')
    
    #проверяем accuracy на валидационной выборке и ищем лучшее n
    df_valid['predictions'] += nu * tree.predict(features_valid)
    real_preds_valid = (df_valid['predictions']+0.5).astype('int')
    acc_valid = accuracy_score(real_preds_valid, target_valid)
    if acc_valid > best_acc:
        best_acc = acc_valid
        best_n = i
print('Наилучшее accuracy по валидационной выборке наблюдается на шаге', best_n, ' и равно', round(best_acc, 3))


AttributeError: 'DecisionTreeRegressor' object has no attribute 'plot_tree'

Теперь сделаем все заново, но возьмем количество деревьев n = 125 (+1) и посмотрим, как модель покажет себя на тестовой выборке.

In [12]:
df_train['predictions'] = df_train['is_ultra'].mean()
df_valid['predictions'] = df_valid['is_ultra'].mean()
n = 126
nu = 0.1
best_acc = 0
trees = []
for i in range(n):
    df_train['residuals'] = df_train['is_ultra'] - df_train['predictions']
    tree = DecisionTreeRegressor(max_depth = 1)
    tree.fit(features_train, df_train['residuals'])
    df_train['predictions'] += nu * tree.predict(features_train)
    real_preds_train = (df_train['predictions']+0.5).astype('int')
    
    #проверяем accuracy на валидационной выборке и ищем лучшее n
    df_valid['predictions'] += nu * tree.predict(features_valid)
    real_preds_valid = (df_valid['predictions']+0.5).astype('int')
    acc_valid = accuracy_score(real_preds_valid, target_valid)
    trees.append(tree)
    if acc_valid > best_acc:
        best_acc = acc_valid
        best_n = i
print('Наилучшее accuracy по валидационной выборке наблюдается на шаге', best_n, ' и равно', round(best_acc, 3))

Наилучшее accuracy по валидационной выборке наблюдается на шаге 125  и равно 0.778


In [13]:
df_test['predictions'] = df_test['is_ultra'].mean()
for tree in trees:
    df_test['predictions'] += nu * tree.predict(features_test)
real_preds_test = (df_test['predictions']+0.5).astype('int')
acc_test = accuracy_score(real_preds_test, target_test)
print('Accuracy по тестовой выборке на получившейся модели градиентного бустинга над решающими деревьями:', round(acc_test, 3))

Accuracy по тестовой выборке на получившейся модели градиентного бустинга над решающими деревьями: 0.796


### Кажется, у меня получилось!
<a id='step_15'></a>
Да, градиентный бустинг над решающими деревьями дал лучший результат по тестовой выборке, хотя по валидационной немного уступает. Тест на адекватность тоже проходим, поскольку `accuracy` лучшего константного предсказания меньше примерно на 10%.

## Подводим итоги
<a id='step_16'></a>
Мы "покрутили" 4 разные модели и оценили точность их предсказаний посредством accuracy_score(). Наилушие результаты показали решеющее дерево, случайный лес и модель, получившаяся в результате градиентного бустинга над решающими деревьями. Обе модели прошли проверку на адекватность. <br>
Однако существует противоречие между accuracy по валидационной и тестовой выборкам. По валидационной выигрывает случайный лес, по тестовой - бустинг. Если взять во внимание такой показатель как скорость, то случаный лес проигрывает двум другим. Составим матрицу с плюсами и минусами?

In [14]:
score_matrix = pd.DataFrame(columns = ['Дерево', 'Лес', 'Бустинг'], index = ['Accuracy по валидационной', 'Accuracy по тестовой', 'Скорость'], data = [['-', '+', '-'], ['-', '-', '+'], ['+', '-', '+']])         
score_matrix 

,Дерево,Лес,Бустинг
Accuracy по валидационной,-,+,-
Accuracy по тестовой,-,-,+
Скорость,+,-,+


Так, мы получаем победителя: Градиентный бустинг над решающими деревьями! Модель записана в `trees`.<br>
<br>
P.s. Но если мы сравним дерево и бустинг, будет ничья, так как у дерева лучше `accuracy` по валидационной выборке, пусть разница и незначительна.